In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile",
    
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.adidas-group.com/adidas/job/Gurgaon-Senior-Specialist-Franchise-Analytics-EM-HR/1101901101/?feedId=301201&utm_source=j2w")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.















Senior Specialist Franchise Analytics EM Job Details | adidas

































                    By continuing to use and navigate this website, you are agreeing to the use of cookies.
                
            

                Accept

                Close




Press Tab to Move to Skip to Content Link
Skip to main content








Prepare for interview
Search Jobs
SEE CATEGORIES




















Search by Keyword




Search by Location







                                 
                            















Prepare for interview
Search Jobs
SEE CATEGORIES









View Profile




Employee Login






















Search by Keyword




Search by Location






Show More Options





Loading...







                                            Team:
                                        


All





                                            Location
                                        


All





                              

In [9]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
             """
             ### SCRAPED TEXT FROM WEBSITE:
             {page_data}
             ### INSTRUCTION:
             The scraped text is from the careeer's page of a website.
             Your job is to extract the job postings and return them in JSON format containing the 
             following keys: 'role','experience','skills' and 'description'.
             Only return the valid JSON.
             ### VALID JSON (NO PREMABLE):
             """
    
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Specialist Franchise Analytics EM',
 'experience': '3+ years of professional experience, with extensive experience in the area of Analytics',
 'skills': ['Excellent analytical skills',
  'Good communication and interpersonal skill',
  'Good understanding of retail or consumer business process',
  'SAP, BI visualization tools (e.g. MicroStrategy, Tableau, PowerBI) experience',
  'Understanding of Finance, Customer Service, Sales',
  'High level of attention to detail and quality',
  'Independent, logic and proactive thinker',
  'Ability to work in a highly matrixed organization',
  'Excellent project management skills',
  'Proficient in MS Office, MSS. Alteryx, R or Python'],
 'description': 'Drive sales efficiency and productivity by accurate reporting in the Franchise channel, providing constructive business insights. Build and create cutting edge reporting and analytics services to all stakeholders, enabling them to take quantitatively based decisions and optimize pe

In [11]:
type(json_res)

dict

In [12]:
import pandas as pd

df = pd.read_csv("my_portfolio_database.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [14]:
import chromadb
import uuid 
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [16]:
links = collection.query(query_texts=["Experience in Python","Expertise in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [19]:
job

{'role': 'Senior Specialist Franchise Analytics EM',
 'experience': '3+ years of professional experience, with extensive experience in the area of Analytics',
 'skills': ['Excellent analytical skills',
  'Good communication and interpersonal skill',
  'Good understanding of retail or consumer business process',
  'SAP, BI visualization tools (e.g. MicroStrategy, Tableau, PowerBI) experience',
  'Understanding of Finance, Customer Service, Sales',
  'High level of attention to detail and quality',
  'Independent, logic and proactive thinker',
  'Ability to work in a highly matrixed organization',
  'Excellent project management skills',
  'Proficient in MS Office, MSS. Alteryx, R or Python'],
 'description': 'Drive sales efficiency and productivity by accurate reporting in the Franchise channel, providing constructive business insights. Build and create cutting edge reporting and analytics services to all stakeholders, enabling them to take quantitatively based decisions and optimize pe

In [17]:
job= json_res
job['skills']

['Excellent analytical skills',
 'Good communication and interpersonal skill',
 'Good understanding of retail or consumer business process',
 'SAP, BI visualization tools (e.g. MicroStrategy, Tableau, PowerBI) experience',
 'Understanding of Finance, Customer Service, Sales',
 'High level of attention to detail and quality',
 'Independent, logic and proactive thinker',
 'Ability to work in a highly matrixed organization',
 'Excellent project management skills',
 'Proficient in MS Office, MSS. Alteryx, R or Python']

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Prashant Chandra, a Business Development Executive at AlgoSphere, an AI & Software Consulting 
        company specializing in optimizing business processes through automation. Your firm helps enterprises 
        enhance scalability, streamline operations, reduce costs, and improve overall efficiency through tailored 
        solutions.
        Write a cold email to a prospective client for the job mentioned earlier  explaining how AlgoSphere can meet their needs. 
        Mention the company's expertise in delivering customized data analytics and automation solutions, and 
        highlight the most relevant projects from AlgoSphere's portfolio using the links provided ({link_list}). 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Data-Driven Insights for Your Franchise Analytics with AlgoSphere

Dear Hiring Manager,

I came across the Senior Specialist Franchise Analytics EM role at your organization and was impressed by the emphasis on driving sales efficiency and productivity through accurate reporting and analytics. As a Business Development Executive at AlgoSphere, I'd like to introduce you to our expertise in delivering customized data analytics and automation solutions that can help you achieve your goals.

Our team at AlgoSphere has extensive experience in developing cutting-edge reporting and analytics services using tools like SAP, BI visualization tools (e.g., MicroStrategy, Tableau, PowerBI), and programming languages like Python and R. We've successfully implemented projects that involve data analysis, visualization, and automation, which have resulted in significant improvements in sales efficiency and productivity for our clients.

Some of our notable projects include:

* Developin